In [26]:
!apt-get -qq install -y libopencv-dev


In [27]:
%%writefile rgb_to_gray.cu

#include <stdio.h>
#include <opencv2/opencv.hpp>

__global__ void rgb_to_gray_kernel(unsigned char* inputImage, unsigned char* outputImage, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int idx = (y * width + x) * 3;
        unsigned char r = inputImage[idx];
        unsigned char g = inputImage[idx + 1];
        unsigned char b = inputImage[idx + 2];

        outputImage[y * width + x] = static_cast<unsigned char>(0.299 * r + 0.587 * g + 0.114 * b);
    }
}

void rgb_to_gray(unsigned char* inputImage, unsigned char* outputImage, int width, int height) {
    unsigned char* d_inputImage;
    unsigned char* d_outputImage;

    size_t imageSize = width * height * 3 * sizeof(unsigned char);
    size_t grayImageSize = width * height * sizeof(unsigned char);

    cudaMalloc(&d_inputImage, imageSize);
    cudaMalloc(&d_outputImage, grayImageSize);

    cudaMemcpy(d_inputImage, inputImage, imageSize, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((width + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (height + threadsPerBlock.y - 1) / threadsPerBlock.y);

    rgb_to_gray_kernel<<<numBlocks, threadsPerBlock>>>(d_inputImage, d_outputImage, width, height);

    cudaMemcpy(outputImage, d_outputImage, grayImageSize, cudaMemcpyDeviceToHost);

    cudaFree(d_inputImage);
    cudaFree(d_outputImage);
}

int main() {
    cv::Mat inputImage = cv::imread("/content/download.jpeg", cv::IMREAD_COLOR);
    if (inputImage.empty()) {
        printf("Failed to open the image file\n");
        return -1;
    }

    int width = inputImage.cols;
    int height = inputImage.rows;

    unsigned char* inputImageData = inputImage.data;
    unsigned char* outputImageData = new unsigned char[width * height];

    rgb_to_gray(inputImageData, outputImageData, width, height);

    cv::Mat grayImage(height, width, CV_8UC1, outputImageData);
    cv::imwrite("output_image.jpg", grayImage);

    delete[] outputImageData;

    printf("RGB to grayscale conversion completed.\n");

    return 0;
}


Overwriting rgb_to_gray.cu


In [28]:
!nvcc -o rgb_to_gray rgb_to_gray.cu -I/usr/include/opencv4 -L/usr/lib -lopencv_core -lopencv_highgui -lopencv_imgcodecs

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [29]:
!./rgb_to_gray


RGB to grayscale conversion completed.
